In [9]:
# -----------------------------------------------------------------------------
# Copyright (c) 2022, Lucid Vision Labs, Inc.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES
# OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS
# BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN
# ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.
# -----------------------------------------------------------------------------

In [10]:
import time
from arena_api.system import system

In [11]:
'''
Acquisition: Introduction
    This example introduces the basics of image acquisition. This includes
    setting image acquisition and buffer handling modes, setting the device to
    automatically negotiate packet size, and setting the stream packet resend
    node before starting the image stream. The example then starts acquiring
    images by grabbing and requeuing buffers, and retrieving data on images
    before finally stopping the image stream.
'''

'\nAcquisition: Introduction\n    This example introduces the basics of image acquisition. This includes\n    setting image acquisition and buffer handling modes, setting the device to\n    automatically negotiate packet size, and setting the stream packet resend\n    node before starting the image stream. The example then starts acquiring\n    images by grabbing and requeuing buffers, and retrieving data on images\n    before finally stopping the image stream.\n'

In [12]:
"""
This function waits for the user to connect a device before raising an exception
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
            tries += 1
    else:
        print(f'Created {len(devices)} device(s)')
        device = devices[0]
        nodemap = device.nodemap
        tl_stream_nodemap = device.tl_stream_nodemap
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')


Created 1 device(s)


In [13]:
'''
Store initial settings
    For all settings we modify, we'll change them bace once we have finished
    the example. In this case, just AcquisitionMode
'''
print(f"Store initial values")
initial_acquisition_mode = nodemap.get_node("AcquisitionMode").value
width_initial = nodemap.get_node("Width").value
height_initial = nodemap.get_node("Height").value

Store initial values


In [14]:
'''
Configure device settings
'''

nodemap.get_node("AcquisitionMode").value = "Continuous"
'''
Set buffer handling mode.
    Set buffer handling mode before starting the stream. Starting the
    stream requires the buffer handling mode to be set beforehand. The
    buffer handling mode determines the order and behavior of buffers in
    the underlying stream engine. Setting the buffer handling mode to
    'NewestOnly' ensures the most recent image is delivered, even if it
    means skipping frames.
'''
tl_stream_nodemap["StreamBufferHandlingMode"].value = "NewestOnly"

'''
Enable stream auto negotiate packet size
    Setting the stream packet size is done before starting the stream.
    Setting the stream to automatically negotiate packet size instructs the
    camera to receive the largest packet size that the system will allow.
    This generally increases frame rate and results in fewer interrupts per
    image, thereby reducing CPU load on the host system. Ethernet settings
    may also be manually changed to allow for a larger packet size.
'''
tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True

'''
Enable stream packet resend
    Enable stream packet resend before starting the stream. Images are sent
    from the camera to the host in packets using UDP protocol, which
    includes a header image number, packet number, and timestamp
    information. If a packet is missed while receiving an image, a packet
    resend is requested and this information is used to retrieve and
    redeliver the missing packet in the correct order.
'''
tl_stream_nodemap['StreamPacketResendEnable'].value = True

# Set features before streaming.-------------------------------------------

print(f'Getting \'Width\' and \'Height\' Nodes')
nodes = device.nodemap.get_node(['Width', 'Height'])

# Set width and height to their max values
print('Setting \'Width\' and \'Height\' Nodes value to their '
      'max values')
nodes['Width'].value = nodes['Width'].max
nodes['Height'].value = nodes['Height'].max


Getting 'Width' and 'Height' Nodes
Setting 'Width' and 'Height' Nodes value to their max values


In [15]:
number_of_buffers = 10

'''
Starting the stream allocates buffers, which can be passed in as
    an argument (default: 10), and begins filling them with data.
    Buffers must later be requeued to avoid memory leaks.
'''
device.start_stream(number_of_buffers)
print(f'Stream started with {number_of_buffers} buffers')

'''
Device.get_buffer() returns buffers:
    Device.get_buffer() with no arguments returns one buffer(NOT IN A LIST)
    Device.get_buffer(20) returns 20 buffers(IN A LIST)
'''
print(f'Get {number_of_buffers} buffers in a list')
buffers = device.get_buffer(number_of_buffers)
print("Success")

'''
Print image buffer info
    Buffers contain image data.
    Image data can also be copied and converted using BufferFactory.
    That is necessary to retain image data, as we must also requeue the buffer.
'''
for count, buffer in enumerate(buffers):
    print(f'\tbuffer{count:{2}} received | '
        f'Width = {buffer.width} pxl, '
        f'Height = {buffer.height} pxl, '
        f'Pixel Format = {buffer.pixel_format.name}')

'''
'Device.requeue_buffer()' takes a buffer or many buffers
    in a list or tuple. Failure to requeue can cause
    memory leaks, or running out of buffers
'''
device.requeue_buffer(buffers)
print(f'Requeued {number_of_buffers} buffers')

device.stop_stream()
print(f'Stream stopped')


Stream started with 10 buffers
Get 10 buffers in a list
Success
	buffer 0 received | Width = 2448 pxl, Height = 2048 pxl, Pixel Format = BGR8
	buffer 1 received | Width = 2448 pxl, Height = 2048 pxl, Pixel Format = BGR8
	buffer 2 received | Width = 2448 pxl, Height = 2048 pxl, Pixel Format = BGR8
	buffer 3 received | Width = 2448 pxl, Height = 2048 pxl, Pixel Format = BGR8
	buffer 4 received | Width = 2448 pxl, Height = 2048 pxl, Pixel Format = BGR8
	buffer 5 received | Width = 2448 pxl, Height = 2048 pxl, Pixel Format = BGR8
	buffer 6 received | Width = 2448 pxl, Height = 2048 pxl, Pixel Format = BGR8
	buffer 7 received | Width = 2448 pxl, Height = 2048 pxl, Pixel Format = BGR8
	buffer 8 received | Width = 2448 pxl, Height = 2048 pxl, Pixel Format = BGR8
	buffer 9 received | Width = 2448 pxl, Height = 2048 pxl, Pixel Format = BGR8
Requeued 10 buffers
Stream stopped


In [16]:
'''
Restore initial values and destroy device
'''
nodemap.get_node("AcquisitionMode").value = initial_acquisition_mode
nodemap.get_node("Width").value = width_initial
nodemap.get_node("Height").value = height_initial
'''
Destroy device. This call is optional and will automatically be
    called for any remaining devices when the system module is unloading.
'''
system.destroy_device()
print('Destroyed all created devices')


Destroyed all created devices
